In [1]:
from simulation import Simulation

In [2]:
var={'n':5,
     'p_otu':8, 
     'p_metabolite':4}

In [3]:
s = Simulation(**var)

In [4]:
# otu table, metabolite table
s.run_simulation()

(array([[-0.90629938,  1.63484721,  0.15939034,  3.29388244, -6.68025422],
        [ 0.876755  ,  0.95379279,  0.77018725,  0.40370319,  0.10950536],
        [ 0.55612086,  0.25397307,  0.71811805,  0.47184798,  0.64750057],
        [ 0.1104565 ,  0.69459028,  0.6559263 ,  0.30175756,  0.85188743],
        [ 0.43728581,  0.11582795,  0.26253821,  0.03863883,  0.31329247],
        [ 0.17307246,  0.37017423,  0.48976723,  0.21496795,  0.60186861],
        [ 0.65666739,  0.7194343 ,  0.80471513,  0.55245637,  0.06912267],
        [ 0.03020851,  0.32687223,  0.34824799,  0.01976642,  0.56995405]]),
 array([[ 1.79704713, -1.22358341,  0.38882562, -1.6784882 ,  3.35283689],
        [ 0.60208738,  0.73755021,  0.26880687,  0.33528939,  0.2459522 ],
        [ 0.11142699,  0.6534976 ,  0.45596845,  0.29831462,  0.97240916],
        [ 0.14464215,  0.24815336,  0.34282189,  0.49700364,  0.40938163]]))